In [8]:
import numpy as np

from cmath import rect, exp, polar
from math import radians, acos, inf

CASO = "000"

In [11]:
# Fonte de tensão do equivalente de Thevenin do nó da subestação
VthA = 13800 / (3 ** 0.5) + 0j
#VthB = VthA * rect(1, radians(-120))
#VthC = VthA * rect(1, radians(120))

# Impedância do equivalente de Thevenin do nó da subestação
Zth = (VthA ** 2)
Zth /= (10e8 / 3) * exp(-radians(80) * 1j)

IthA = VthA / Zth
#IthB = VthB / Zth
#IthC = VthC / Zth

Yth = 1 / Zth

Rmax = 10

exec(open(f"data/CAR{CASO}.py", "r").read())
exec(open(f"data/TOP{CASO}.py", "r").read())
exec(open(f"data/VOL{CASO}.py", "r").read())

cargas    = np.array(cargas)
Emedido   = np.array(Emedido)
topologia = np.array(topologia)

Zcarga = np.array([0 + 0j] * len(cargas))

In [16]:
#resultado = np.array([])
#resposta  = np.array([])

#ultima_posicao = len(topologia) + 2
#ultima_posicao *= 3

for caso in range(len(Emedido)):
    E10meas = Emedido[caso, 1] + 1j * Emedido[caso, 2]
    
    for i in range(len(cargas)):
        Zcarga[i] = (13800 ** 2)
        Zcarga[i] /= 1000 * complex(cargas[i, 1], -cargas[i, 1] * tan(acos(cargas[i, 2])))
        
    # Teste do trecho 10-20
    # 1) Inicialização da função a ser otimizada
    funcao = np.inf
    for rf in np.arange(0.1, Rmax + 0.05, 0.1):
        for x in np.arange(1, topologia[0, 2]):
            # 2) Montagem da matriz Ybus
            # Posição 1 ( linha e/ou coluna da matriz ) -> equivale ao nó 10
            # Posição 2 ( linha e/ou coluna da matriz ) -> equivale ao nó 20
            # Posição 3 ( linha e/ou coluna da matriz ) -> equivale ao nó 30
            # Posição 4 ( linha e/ou coluna da matriz ) -> equivale ao nó 40
            # Posição 5 ( linha e/ou coluna da matriz ) -> equivale ao nó de falta
  
            dist1 = topologia[0, 2]
            raa1 = topologia[0, 3]
            xaa1 = topologia[0, 4]
            rxaa1 = raa1 + 1j * xaa1
            
            dist2 = topologia[1, 2]
            raa2 = topologia[1, 3]
            xaa2 = topologia[1, 4]
            rxaa2 = raa2 + 1j * xaa2
            
            dist3 = topologia[2, 2]
            raa3 = topologia[2, 3]
            xaa3 = topologia[2, 4]
            rxaa3 = raa3 + 1j * xaa3
            
            Ybus = np.zeros((5, 5), complex)
            Ybus[0, 0] = Yth + 1 / (x * rxaa1)
            Ybus[0, 1] = 0
            Ybus[0, 2] = 0
            Ybus[0, 3] = 0
            Ybus[0, 4] = -1 / (x * rxaa1)
                
            Ybus[1, 0] =  Ybus[0, 1] 
            Ybus[1, 1] =  1 / ((dist1 - x) * rxaa1) + 1 / Zcarga[0] + 1 / (dist2 * rxaa2) + 1 / (dist3 * rxaa3)
            Ybus[1, 2] = -1 / (dist2 * rxaa2)
            Ybus[1, 3] = -1 / (dist3 * rxaa3)
            Ybus[1, 4] = -1 / ((dist1 - x) * rxaa1)
            
            Ybus[2, 0] = Ybus[0, 2]
            Ybus[2, 1] = Ybus[1, 2]
            Ybus[2, 2] = 1 / Zcarga[1] + 1 / (dist2 * rxaa2)
            Ybus[2, 3] = 0
            Ybus[2, 4] = 0
            
            Ybus[3, 0] = Ybus[0, 3]
            Ybus[3, 1] = Ybus[1, 3]
            Ybus[3, 2] = Ybus[2, 3]
            Ybus[3, 3] = 1 / Zcarga[2] + 1 / (dist3 * rxaa3)
            Ybus[3, 4] = 0
            
            Ybus[4, 0] = Ybus[0, 4]
            Ybus[4, 1] = Ybus[1, 4]            
            Ybus[4, 2] = Ybus[2, 4]            
            Ybus[4, 3] = Ybus[3, 4]
            Ybus[4, 4] = 1 / (x * rxaa1) + 1 / rf + 1 / ((dist1 - x) * rxaa1)      
            
            # 3) Cálculo das tensões nodais
            E       = np.linalg.solve(Ybus, np.array([[IthA], [0], [0], [0], [0]]))
            E10calc = E[0, 0]
            
            funcao_old = abs(E10meas - E10calc)/abs(E10meas)
            if funcao_old < funcao:
                xcalc = x
                rcalc = rf
                funcao = funcao_old
    print(f"{caso + 1:03d}, 10, 20, {xcalc:.2f}, {rcalc:.2f}, {funcao:.5f}")

001, 10, 20, 49.00, 9.40, 0.00068
002, 10, 20, 49.00, 7.30, 0.00060
003, 10, 20, 10.00, 1.80, 0.00119
004, 10, 20, 49.00, 2.80, 0.00051
005, 10, 20, 49.00, 1.10, 0.00232
006, 10, 20, 49.00, 3.10, 0.00104
007, 10, 20, 49.00, 4.20, 0.00289
008, 10, 20, 49.00, 7.00, 0.00179
009, 10, 20, 49.00, 4.50, 0.00198
010, 10, 20, 49.00, 6.10, 0.00219


In [13]:
Zth

(0.033069558954890235+0.18754678848364492j)

In [1]:
Emedido[0]

NameError: name 'Emedido' is not defined